# Imports

In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from qiskit.synthesis import synth_clifford_depth_lnn
from qiskit.quantum_info import random_unitary
from qiskit.circuit.library import UnitaryGate
from qiskit import transpile
import numpy as np
from qiskit.transpiler import OptimizationMetric

# Global Methods

In [100]:
def get_t_count(circuit):
    """Count the number of T and T† gates in a quantum circuit."""
    t_count = 0
    for instr, _, _ in circuit:
        if instr.name in ['t', 'tdg']:
            t_count += 1
    return t_count

## gridsynth

In [182]:
def apply_gridsynth_to_circuit(circuit, qubit, gridsynth_string):
    """
    Apply a gridsynth rotation sequence to an existing circuit.

    Args:
    circuit: Existing QuantumCircuit
    qubit_index: Which qubit to apply gates to
    gridsynth_string: Output string from gridsynth
    """
    # Apply gates from right to left
    for gate in reversed(gridsynth_string.upper()):
        if gate == 'T':
            circuit.t(qubit)
        elif gate == 'S':
            circuit.s(qubit)
        elif gate == 'H':
            circuit.h(qubit)
        elif gate == 'X':
            circuit.h(qubit)
            circuit.s(qubit)
            circuit.s(qubit)
            circuit.h(qubit)
            # circuit.x(qubit)
            # pass
        elif gate == 'W':
            circuit.global_phase += np.pi/4
            # pass
    return circuit  

In [136]:
import subprocess

def run_gridsynth(angle, random_seed=40) -> str:
    """
    Run gridsynth and return the output string.
    
    Args:
        angle: Angle expression as string (e.g., "pi/4", "-2*pi/7")
    
    Returns:
        Gridsynth output string
    """
    result = subprocess.run(
        [r"C:\cabal\bin\gridsynth.exe", angle, f"-r {random_seed}"],
        capture_output=True,
        text=True
    )
    return result.stdout.strip()

def run_gridsynth(angle, options=None) -> str:
    """
    Run gridsynth and return the output string.
    
    Args:
        angle: Angle expression as string (e.g., "pi/4", "-2*pi/7")
        options: List of option strings (e.g., ["-d", "100"])
    
    Returns:
        Gridsynth output string
    """
    cmd = [r"C:\cabal\bin\gridsynth.exe"]
    if options:
        cmd.extend(options)
    cmd.append(angle)
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    return result.stdout.strip()


### Testing gridsynth

In [94]:
run_gridsynth("pi/4")  

'HTHTSHTHTHTSHTHTHTSHTSHTHTSHTHTHTHTHTSHTSHTHTSHTSHTSHTSHTSHTSHTHTHTHTSHTSHTHTSHTHTSHTHTSHTSHTHTSHTSHTHTSHTHTHTHTHTSHTSHTHTHTHTSHTHTHTHTHTHTHTHTSHTHTSHTHTSHTSHTSHTSHTHTSHTSHTSHTHTHTSHTHTHTHTHTHTHTHTSHTSHTHTSHTHTSHTSHTSHTSHTHTHTSHTSHTHTSHTHTHTSHTHTHTHTSHTSHTSHTHTSHTSHTHTHTSHTHTSHTHTHTSHTHTSHTSHTHTSHTSHTHTSHTSHTHTSHTSHTSHSSS'

In [ ]:
qc = QuantumCircuit(1)
qc.rz(np.pi / 4, 0)
u = Operator(qc)

In [52]:
gridsynth_output = "HTHTSHTSHTSHTHTSHTSHTHTSHTHTHTHTHTHTSHTHTSHTHTHTSHTSHTSHTHTHTSHTSHTHTHTSHTHTHTSHTSHTHTSHTHTSHTHTHTHTSHTSHTHTSHTHTHTHTSHTHTSHTSHTHTHTSHTSHTSHTHTHTSHTHTHTHTSHTHTSHTSHTSHTHTSHTHTSHTSHTHTSHTHTSHTSHTHTHTHTSHTSHTHTSHTHTHTSHTSHTHTHTSHTHTHTHTHTSHTHTSHTSHTSHTHTHTHTHTHTSHTHTSHTSHTSHTSHTHTSHTSHTSHTSHTHTHTHTHTSHTHTSHTHTHTSHTHTHXSSWWWWWW"

# Create a quantum circuit
qc = QuantumCircuit(1)

# Apply the gridsynth sequence to qubit 0
grid_synth_qc = apply_gridsynth_to_circuit(qc, 0, gridsynth_output)

# grid_synth_qc.draw()

u_gridsynth = Operator(grid_synth_qc)

In [57]:
u_gridsynth.equiv(u)  

True

# Challenge 1

In [96]:
# Challenge 1

qc = QuantumCircuit(2)

# Controlled-Y: control qubit 0, target qubit 1
qc.cy(0, 1)

optimizationMetric = OptimizationMetric(2)

# 3. Transpile to H, T, T†, CNOT
transpiled = transpile(
    qc,
    basis_gates=['h', 't', 'tdg', 'cx', 's', 'sdg'],
    optimization_level=3,
    
)

# 4. Count T gates
t_count = transpiled.count_ops().get('t', 0) + \
          transpiled.count_ops().get('tdg', 0)

print(f"T-count: {t_count}")



print(qc.draw())
print(transpiled.draw())

T-count: 0
          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ Y ├
     └───┘
                              
q_0: ─────────────────■───────
     ┌───┐┌───┐┌───┐┌─┴─┐┌───┐
q_1: ┤ S ├┤ S ├┤ S ├┤ X ├┤ S ├
     └───┘└───┘└───┘└───┘└───┘


## Testing challenge 1

In [85]:
qc2 = QuantumCircuit(2)
qc2.tdg(1)
qc2.tdg(1)
qc2.cx(0, 1)
qc2.t(1)
qc2.t(1)

print(qc2.draw())

                                  
q_0: ────────────────■────────────
     ┌─────┐┌─────┐┌─┴─┐┌───┐┌───┐
q_1: ┤ Tdg ├┤ Tdg ├┤ X ├┤ T ├┤ T ├
     └─────┘└─────┘└───┘└───┘└───┘


In [86]:
op1 = Operator(qc)
op2 = Operator(qc2)
print(op1.equiv(op2))
op1.equiv(transpiled)

True


True

In [89]:
ts = QuantumCircuit(1)
ts.t(0)
ts.t(0)
ts.t(0)
ts.t(0)
ts.t(0)
ts.t(0)
ts_op = Operator(ts)

sdgs = QuantumCircuit(1)
sdgs.sdg(0)
sdgs_op = Operator(sdgs)

ts_op.equiv(sdgs_op)

True

In [87]:
stop

NameError: name 'stop' is not defined

# Challenge 2

In [ ]:
qc_2 = QuantumCircuit(2)
qc_2.cx(0, 1)
qc_2.s(1)
qc_2.h(1)
qc_2 = apply_gridsynth_to_circuit(qc_2, 1, run_gridsynth("pi/7", ["-d", "100"]))  # qc_2.ry(np.pi/7, 1)
qc_2.h(1)
qc_2.sdg(1)
qc_2.cx(0, 1)

# print(qc_2.draw())
qc_2_op = Operator(qc_2) 


u_target = QuantumCircuit(2)
u_target.cry(np.pi/7, 0, 1)
u_target_op = Operator(u_target)

print(u_target_op.equiv(qc_2_op))
print(u_target_op)
print(qc_2_op)


print(qc_2_op.data - u_target_op.data)
np.linalg.norm(qc_2_op.data - u_target_op.data)


False
Operator([[ 1.        +0.j,  0.        +0.j,  0.        +0.j,
            0.        +0.j],
          [ 0.        +0.j,  0.97492791+0.j,  0.        +0.j,
           -0.22252093+0.j],
          [ 0.        +0.j,  0.        +0.j,  1.        +0.j,
            0.        +0.j],
          [ 0.        +0.j,  0.22252093+0.j,  0.        +0.j,
            0.97492791+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Operator([[ 0.97492791+0.97492791j,  0.        +0.j        ,
            0.22252093+0.22252093j,  0.        +0.j        ],
          [ 0.        +0.j        ,  0.97492791+0.97492791j,
            0.        +0.j        , -0.22252093-0.22252093j],
          [-0.22252093-0.22252093j,  0.        +0.j        ,
            0.97492791+0.97492791j,  0.        +0.j        ],
          [ 0.        +0.j        ,  0.22252093+0.22252093j,
            0.        +0.j        ,  0.97492791+0.97492791j]],
         input_dims=(2, 2), output_dims=(2, 2))
[[-2.50720878e-02+0.97492791j  0.0000000

np.float64(2.0249168751481963)

In [184]:
1/0.97492791 

1.0257168655680398

# Challenge 3

In [119]:
qc_3 = QuantumCircuit(2)
qc_3.cx(0, 1)
qc_3 = apply_gridsynth_to_circuit(qc_3, 1, run_gridsynth("0-2*pi/7")) # qc_3.rz(-2*np.pi/7, 1)
qc_3.cx(0, 1)

qc_3.draw()


global phase: 3π/2
                                                                           »
q_0: ──■───────────────────────────────────────────────────────────────────»
     ┌─┴─┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
q_1: ┤ X ├┤ X ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├»
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q_1: ┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├»
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q_1: ┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├»
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q_1: ┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├»
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q_1: ┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├»
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q_1: ┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├»
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q_1: ┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├»
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q_1: ┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├»
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q_1: ┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├»
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q_1: ┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├┤ S ├┤ T ├┤ H ├»
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──

# Challenge 4

In [169]:
qc_4 = QuantumCircuit(2)
qc_4.h([0, 1])
qc_4.cx(0, 1)
qc_4 = apply_gridsynth_to_circuit(qc_4, 1, run_gridsynth("0-2*pi/7")) # qc_4.rz(-2 * np.pi / 7, 1)
qc_4.cx(0, 1)
qc_4.h([0, 1])
qc_4.sdg([0,1])
qc_4.h([0, 1])
qc_4 = apply_gridsynth_to_circuit(qc_4, 1, run_gridsynth("0-2*pi/7")) # qc_4.rz(-2 * np.pi / 7, 1)
qc_4.h([0, 1])
qc_4.s([0,1])

print(qc_4.draw())

# transpiled_4 = transpile(
#     qc_4,
#     basis_gates=['h', 't', 'tdg', 'cx', 's', 'sdg',],
#     optimization_level=3
# )
# t_count_4 = get_t_count(transpiled_4)
# print(f"T-count for qc_4_transpiled: {t_count_4}")
# # print(transpiled_4.draw())

global phase: 7π/4
     ┌───┐                                                                 »
q_0: ┤ H ├──■──────────────────────────────────────────────────────────────»
     ├───┤┌─┴─┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
q_1: ┤ H ├┤ X ├┤ S ├┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├»
     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q_1: ┤ S ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ S ├»
«     └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«                                                                           »
«q_0: ──────────────────────────────────────────────────────────────────────»
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───

# Challenge 6

In [111]:
qc_6 = QuantumCircuit(3)
qc_6.h(0)
qc_6.h(1)
qc_6.cx(0, 1)
qc_6.t(1)
qc_6.cx(0, 1)
qc_6.h(0)
qc_6.h(1)
qc_6.rz(np.pi/7, 0)
qc_6.rz(np.pi/7, 1)

transpiled_6 = transpile(
    qc_6,
    basis_gates=['h', 't', 'tdg', 'cx', 's', 'sdg'],
    optimization_level=3,
)

print(qc_6.draw())

print(get_t_count(transpiled_6))

print(transpiled_6.draw())


     ┌───┐               ┌───┐┌─────────┐
q_0: ┤ H ├──■─────────■──┤ H ├┤ Rz(π/7) ├
     ├───┤┌─┴─┐┌───┐┌─┴─┐├───┤├─────────┤
q_1: ┤ H ├┤ X ├┤ T ├┤ X ├┤ H ├┤ Rz(π/7) ├
     └───┘└───┘└───┘└───┘└───┘└─────────┘
q_2: ────────────────────────────────────
                                         
17945


C:\Users\marlo_2lbtn5r\AppData\Local\Temp\ipykernel_10876\339081247.py:4: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for instr, _, _ in circuit:


global phase: π/2
     ┌───┐               ┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌─────┐┌───┐┌─────┐»
q_0: ┤ H ├──■─────────■──┤ Tdg ├┤ H ├┤ T ├┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├┤ Sdg ├»
     ├───┤┌─┴─┐┌───┐┌─┴─┐├─────┤├───┤├───┤├───┤├─────┤├───┤├─────┤├───┤├─────┤»
q_1: ┤ H ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ H ├┤ T ├┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├┤ Sdg ├»
     └───┘└───┘└───┘└───┘└─────┘└───┘└───┘└───┘└─────┘└───┘└─────┘└───┘└─────┘»
q_2: ─────────────────────────────────────────────────────────────────────────»
                                                                              »
«     ┌───┐┌─────┐┌───┐┌───┐┌───┐┌───┐┌─────┐┌─────┐┌───┐┌─────┐┌───┐┌───┐┌───┐»
«q_0: ┤ H ├┤ Sdg ├┤ T ├┤ S ├┤ H ├┤ S ├┤ Tdg ├┤ Sdg ├┤ H ├┤ Sdg ├┤ T ├┤ S ├┤ H ├»
«     ├───┤├─────┤├───┤├───┤├───┤├───┤├─────┤├─────┤├───┤├─────┤├───┤├───┤├───┤»
«q_1: ┤ H ├┤ Sdg ├┤ T ├┤ S ├┤ H ├┤ S ├┤ Tdg ├┤ Sdg ├┤ H ├┤ Sdg ├┤ T ├┤ S ├┤ H ├»
«     └───┘└─────┘└───┘└───┘└───┘└───┘└─────┘└─────┘└───┘└─────┘└───┘└───┘└───┘»
«q_2: ───────────

# Challenge 8

In [14]:
U_8 = 1/2 * np.array([[1, 1, 1, 1],
                    [1, 1j, -1, -1j],
                    [1, -1, 1, -1],
                    [1, -1j, -1, 1j]])
U_8_gate = UnitaryGate(U_8)
qc_10 = QuantumCircuit(2)
qc_10.append(U_8_gate, [0, 1])
transpiled_10 = transpile(
    qc_10,
    basis_gates=['h', 't', 'tdg', 'cx', 's', 'sdg'],
    optimization_level=3,
)
print(qc_10.decompose().draw())
print(get_t_count(transpiled_10))
# print(transpiled_10.draw())

global phase: 1.5112
     ┌──────────────────┐          ┌─────────────────┐          »
q_0: ┤ U(π/2,-π/2,-π/2) ├──■───────┤ U(π/2,-π/2,π/2) ├───────■──»
     ├──────────────────┤┌─┴─┐┌────┴─────────────────┴────┐┌─┴─┐»
q_1: ┤ U(2.6887,π/2,-π) ├┤ X ├┤ U(1.3782,0.41245,0.41245) ├┤ X ├»
     └──────────────────┘└───┘└───────────────────────────┘└───┘»
«          ┌───────────────┐              ┌────────────────┐  
«q_0: ─────┤ U(π/4,-π,π/2) ├────────■─────┤ U(π/2,π/4,π/2) ├──
«     ┌────┴───────────────┴─────┐┌─┴─┐┌──┴────────────────┴─┐
«q_1: ┤ U(1.9752,2.9318,-2.0668) ├┤ X ├┤ U(1.2383,-π/2,-π/2) ├
«     └──────────────────────────┘└───┘└─────────────────────┘


NameError: name 'get_t_count' is not defined

## Version based on "Teleportation-based quantum homomorphic encryption scheme with quasi-compactness and perfect security"

In [ ]:
qc_8_v2 = QuantumCircuit(2)
# qc_8_v2.h(0)
# qc_8_v2.cx(0, 1)
# qc_8_v2.tdg(0)
# qc_8_v2.cx(0, 1)
# qc_8_v2.t([0,1])
# qc_8_v2.h(1)

# qc_8_v2.h(1)
# qc_8_v2.t([0,1])
# qc_8_v2.cx(0, 1)
# qc_8_v2.tdg(0)
# qc_8_v2.cx(0, 1)
# qc_8_v2.h(0)


print(qc_8_v2.draw())
u = Operator(qc_8_v2)

print(u.equiv(U_8))
print(u)
print(U_8)

     ┌───┐          ┌─────┐     ┌───┐
q_0: ┤ T ├───────■──┤ Tdg ├──■──┤ H ├
     ├───┤┌───┐┌─┴─┐└─────┘┌─┴─┐└───┘
q_1: ┤ H ├┤ T ├┤ X ├───────┤ X ├─────
     └───┘└───┘└───┘       └───┘     
False
Operator([[ 0.5       +0.j        ,  0.5       +0.j        ,
            0.5       +0.j        ,  0.5       +0.j        ],
          [ 0.5       +0.j        , -0.5       +0.j        ,
            0.5       +0.j        , -0.5       +0.j        ],
          [ 0.35355339+0.35355339j,  0.35355339+0.35355339j,
           -0.35355339-0.35355339j, -0.35355339-0.35355339j],
          [ 0.35355339+0.35355339j, -0.35355339-0.35355339j,
           -0.35355339-0.35355339j,  0.35355339+0.35355339j]],
         input_dims=(2, 2), output_dims=(2, 2))
[[ 0.5+0.j   0.5+0.j   0.5+0.j   0.5+0.j ]
 [ 0.5+0.j   0. +0.5j -0.5+0.j   0. -0.5j]
 [ 0.5+0.j  -0.5+0.j   0.5+0.j  -0.5+0.j ]
 [ 0.5+0.j   0. -0.5j -0.5+0.j   0. +0.5j]]


In [114]:
qc_8_v3 = QuantumCircuit(2)
qc_8_v3.h(0)
qc_8_v3.csdg(0, 1)
qc_8_v3.h(1)


print(qc_8_v3.draw())
u = Operator(qc_8_v3)
u 
U_8 

print(u.equiv(U_8))
print(u)
print(U_8)

     ┌───┐            
q_0: ┤ H ├───■────────
     └───┘┌──┴──┐┌───┐
q_1: ─────┤ Sdg ├┤ H ├
          └─────┘└───┘
False
Operator([[ 0.5+0.j ,  0.5+0.j ,  0.5+0.j ,  0.5+0.j ],
          [ 0.5+0.j , -0.5+0.j ,  0. -0.5j,  0. +0.5j],
          [ 0.5+0.j ,  0.5+0.j , -0.5+0.j , -0.5+0.j ],
          [ 0.5+0.j , -0.5+0.j ,  0. +0.5j,  0. -0.5j]],
         input_dims=(2, 2), output_dims=(2, 2))
[[ 0.5+0.j   0.5+0.j   0.5+0.j   0.5+0.j ]
 [ 0.5+0.j   0. +0.5j -0.5+0.j   0. -0.5j]
 [ 0.5+0.j  -0.5+0.j   0.5+0.j  -0.5+0.j ]
 [ 0.5+0.j   0. -0.5j -0.5+0.j   0. +0.5j]]


# Challenge 10

In [99]:
# Challenge 10

U = random_unitary(4, seed=42)  # unitary from challenge 10
gate = UnitaryGate(U)

qc_10 = QuantumCircuit(2)
qc_10.append(gate, [0,1])

# 3. Transpile to H, T, T†, CNOT
transpiled_10 = transpile(
    qc_10,
    basis_gates=['h', 't', 'tdg', 'cx', 's', 'sdg',],
    optimization_level=3
)

# 4. Count T gates
t_count = transpiled_10.count_ops().get('t', 0) + \
          transpiled_10.count_ops().get('tdg', 0)

print(f"T-count: {t_count}")


T-count: 68738


# Export circuits to QASM 2

In [120]:
from qiskit import qasm2
qasm2.dump(qc_3, "QASM Files/qc_3.qasm")